In [1]:
import os

import numpy as np
import pandas as pd
import tensorflow_hub as hub
from google.cloud import storage
from tensorflow.keras.models import model_from_json

2022-03-30 12:10:53.572161: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-03-30 12:10:53.572207: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:

client = storage.Client()

# modelName = "GG_model_20211013_6"
# json_file = open(f"{modelName}.json", "r")
# loaded_model_json = json_file.read()
# json_file.close()
# model = model_from_json(loaded_model_json, custom_objects={'KerasLayer': hub.KerasLayer})
# model.load_weights(f"{modelName}_bestweights.hdf5")


In [10]:
files = []
# bucket_name/cashews/Benin_Largearea/benin_prediction_fid_folder
for blob in client.list_blobs(
    'bucket_name', prefix='seasonal_crops_farm_Africa_data/joined_csvs/Farm_Africa_joined_2018_'
):
    files.append(blob.name)
    

In [13]:
for file_ in files:
    file_name = file_.split("/")[-1]
    print (file_name)
    fn = int(file_name.split("_")[-1].split(".")[0])
    print (fn)
    if fn in range(70):
        if not os.path.isfile(file_name):
            os.system(f"gsutil cp gs://bucket_name/{file_} .")
        else:
            print(f"File {file_name} already on machine")

        df = pd.read_csv(file_name)
#         data_df = df.drop(columns=["row_loc","col_loc", "tile", "fid"])
        for i in range(8) :
            B08_features =  df.filter(regex='._B08')
            B08_features_col = B08_features.columns
#         print(B8A_features_col)
            B04_features =  df.filter(regex='._B04')
            B04_features_col = B04_features.columns
            NDVI = []
            NDVI.append('NDVI_{}'.format(i))
            B08 = '{}'.format(B08_features_col[i])
            B04 = '{}'.format(B04_features_col[i])
            new_df = ((df[B08]-df[B04])/(df[B08]+df[B04]))
            df['NDVI_{}'.format(i)] = (df[B08]-df[B04])/(df[B08]+df[B04])
            df['NDVI_{}'.format(i)] = df['NDVI_{}'.format(i)].fillna(0.000001)
            
#           Nomormalise the data
#             df['NDVI_Standardise_{}'.format(i)] = ((df['NDVI_{}'.format(i)]-(-0.3991874))/(0.9409440-(-0.3991874))*1)
#             df['NDVI_Standardise_{}'.format(i)] = df['NDVI_Standardise_{}'.format(i)].fillna(0.000001)
#         test_data = data_df.to_numpy()
#         max_value = test_data.max()
#         test_data = test_data / max_value
#         test_data = np.reshape(test_data, (test_data.shape[0], int(test_data.shape[1]/12), 12))

#         predictions = model.predict(test_data, verbose=1)

#         pred_df = pd.DataFrame(predictions)
#         pred_df["predicted_class"] = pred_df.idxmax(axis=1)
#         pred_df[["row_loc","col_loc","tile"]] = df[["row_loc","col_loc","tile"]].reset_index(drop=True).astype(int)
        
    
#     Calculating standard deviation
        df['NDVI_stand_dev'] = df[['NDVI_0', 'NDVI_1', 'NDVI_2', 'NDVI_3', 'NDVI_4', 'NDVI_5', 'NDVI_6', 'NDVI_7']].std(axis=1)
        print (df['NDVI_stand_dev'])
    
#         df_veg['NDVI_change'] = (1-((df_veg['NDVI_0'])/(df_veg['NDVI_1'])))
# df['NDVI_{}'.format(i)] = df['NDVI_{}'.format(i)].fillna(0.000001)

# df['non_vegetation'] = df['NDVI_change'].apply(lambda x :1 if x<0.199 else 0)
        df['seasonal_crops'] = df['NDVI_stand_dev'].apply(lambda x :1 if (0.1<x<0.3) else 0)
# df['high_vegetation'] = df['NDVI_change'].apply(lambda x :1 if x>=0.501 else 0)
#         df_veg.head()
#         df1 = df.query('fid > 200000000')
        df = df.drop(columns=[ '_20180116_B02', '_20180116_B03',
       '_20180116_B04', '_20180116_B08', '_20180215_B02', '_20180215_B03',
       '_20180215_B04', '_20180215_B08', '_20180322_B02', '_20180322_B03',
       '_20180322_B04', '_20180322_B08', '_20180511_B02', '_20180511_B03',
       '_20180511_B04', '_20180511_B08', '_20180720_B02', '_20180720_B03',
       '_20180720_B04', '_20180720_B08', '_20180923_B02', '_20180923_B03',
       '_20180923_B04', '_20180923_B08', '_20181003_B02', '_20181003_B03',
       '_20181003_B04', '_20181003_B08', '_20181227_B02', '_20181227_B03',
       '_20181227_B04', '_20181227_B08' ])
    
        df.to_csv(f"NDVI_csvs_2018/NDVI_2018_{file_name.split('_')[-1]}", index=False)

# new_df.to_csv('NDVI_mean_Classification_aoi5.csv')
        
#         print(f"{file_name} already worked on")

        # empty variables
        del df
#         del test_data
#         del pred_df

        os.system(f"rm {file_name}")
    else:
        print(f"{file_name} already worked on")

# """
# # NOTE: Join generated csvs
# files_ = os.listdir("prediction_csvs")
# files_.sort()
# dfs = [pd.read_csv(f"prediction_csvs/{f_}") for f_ in files_]
# df = pd.concat(dfs, ignore_index=True)
# print(f"Check generated unique classes {df['predicted_class'].unique()}")
# df.to_csv("benin_prediction_tile.csv", index=False)
# """


Farm_Africa_joined_2018_000000000000.csv
0
File Farm_Africa_joined_2018_000000000000.csv already on machine
0          0.020995
1          0.054268
2          0.117507
3          0.076917
4          0.000000
             ...   
1793443    0.085949
1793444    0.075375
1793445    0.057328
1793446    0.224558
1793447    0.104125
Name: NDVI_stand_dev, Length: 1793448, dtype: float64
Farm_Africa_joined_2018_000000000001.csv
1


Copying gs://short-term-storage-gecko-bucket/seasonal_crops_farm_Africa_data/joined_csvs/Farm_Africa_joined_2018_000000000001.csv...
==> NOTE: You are downloading one or more large file(s), which would            
run significantly faster if you enabled sliced object downloads. This
feature is enabled by default but requires that compiled crcmod be
installed (see "gsutil help crcmod").

\ [1 files][189.2 MiB/189.2 MiB]                                                
Operation completed over 1 objects/189.2 MiB.                                    


0          0.000000
1          0.000000
2          0.000000
3          0.018617
4          0.000000
             ...   
1790386    0.227894
1790387    0.133258
1790388    0.142916
1790389    0.124207
1790390    0.128476
Name: NDVI_stand_dev, Length: 1790391, dtype: float64
Farm_Africa_joined_2018_000000000002.csv
2


Copying gs://short-term-storage-gecko-bucket/seasonal_crops_farm_Africa_data/joined_csvs/Farm_Africa_joined_2018_000000000002.csv...
==> NOTE: You are downloading one or more large file(s), which would            
run significantly faster if you enabled sliced object downloads. This
feature is enabled by default but requires that compiled crcmod be
installed (see "gsutil help crcmod").

\ [1 files][188.9 MiB/188.9 MiB]                                                
Operation completed over 1 objects/188.9 MiB.                                    


0          0.056113
1          0.055373
2          0.058487
3          0.153425
4          0.000000
             ...   
1787903    0.226788
1787904    0.067382
1787905    0.129135
1787906    0.234682
1787907    0.142201
Name: NDVI_stand_dev, Length: 1787908, dtype: float64
Farm_Africa_joined_2018_000000000003.csv
3


...

0          0.185981
1          0.000000
2          0.033385
3          0.092040
4          0.053554
             ...   
1789596    0.161256
1789597    0.144789
1789598    0.109717
1789599    0.123973
1789600    0.156148
Name: NDVI_stand_dev, Length: 1789601, dtype: float64
Farm_Africa_joined_2018_000000000006.csv
6


Copying gs://short-term-storage-gecko-bucket/seasonal_crops_farm_Africa_data/joined_csvs/Farm_Africa_joined_2018_000000000006.csv...
==> NOTE: You are downloading one or more large file(s), which would            
run significantly faster if you enabled sliced object downloads. This
feature is enabled by default but requires that compiled crcmod be
installed (see "gsutil help crcmod").

| [1 files][189.4 MiB/189.4 MiB]                                                
Operation completed over 1 objects/189.4 MiB.                                    


0          0.067055
1          0.102529
2          0.059288
3          0.135500
4          0.219735
             ...   
1792495    0.199444
1792496    0.133577
1792497    0.126185
1792498    0.144810
1792499    0.106238
Name: NDVI_stand_dev, Length: 1792500, dtype: float64
Farm_Africa_joined_2018_000000000007.csv
7


Copying gs://short-term-storage-gecko-bucket/seasonal_crops_farm_Africa_data/joined_csvs/Farm_Africa_joined_2018_000000000007.csv...
==> NOTE: You are downloading one or more large file(s), which would            
run significantly faster if you enabled sliced object downloads. This
feature is enabled by default but requires that compiled crcmod be
installed (see "gsutil help crcmod").

\ [1 files][189.1 MiB/189.1 MiB]                                                
Operation completed over 1 objects/189.1 MiB.                                    


0          0.000000
1          0.000000
2          0.000000
3          0.000000
4          0.000000
             ...   
1790220    0.125552
1790221    0.181612
1790222    0.106721
1790223    0.172593
1790224    0.098919
Name: NDVI_stand_dev, Length: 1790225, dtype: float64


In [8]:
df.head()

,row_loc,col_loc,tile,fid,NDVI_0,NDVI_1,NDVI_2,NDVI_3,NDVI_4,NDVI_5,NDVI_6,NDVI_7,NDVI_stand_dev,seasonal_crops
0,1914,2724,0,6452904,0.052632,0.042553,0.057471,0.042735,0.039216,0.046729,0.043478,-0.010526,0.020995,0
1,1778,142,0,5992002,0.833333,0.872340,0.859155,0.885714,0.735294,0.857143,0.916667,0.882353,0.054268,1
2,1757,38,0,5921128,0.846154,0.822222,0.826087,0.909091,0.534247,0.838710,0.873016,0.877551,0.117507,1
3,3481,2008,0,11732978,0.150943,0.128205,0.240000,0.254545,0.280702,0.356522,0.315789,0.257732,0.076917,1
4,3325,3366,0,11208616,0.000001,0.000001,0.000001,0.000001,0.000001,0.000001,0.000001,0.000001,0.000000,0


In [12]:
df.columns

Index(['row_loc', 'col_loc', 'tile', 'fid', '_20180116_B02', '_20180116_B03',
       '_20180116_B04', '_20180116_B08', '_20180215_B02', '_20180215_B03',
       '_20180215_B04', '_20180215_B08', '_20180322_B02', '_20180322_B03',
       '_20180322_B04', '_20180322_B08', '_20180511_B02', '_20180511_B03',
       '_20180511_B04', '_20180511_B08', '_20180720_B02', '_20180720_B03',
       '_20180720_B04', '_20180720_B08', '_20180923_B02', '_20180923_B03',
       '_20180923_B04', '_20180923_B08', '_20181003_B02', '_20181003_B03',
       '_20181003_B04', '_20181003_B08', '_20181227_B02', '_20181227_B03',
       '_20181227_B04', '_20181227_B08', 'NDVI_0', 'NDVI_1', 'NDVI_2',
       'NDVI_3', 'NDVI_4', 'NDVI_5', 'NDVI_6', 'NDVI_7', 'NDVI_stand_dev',
       'seasonal_crops'],
      dtype='object')